In [1]:
import time
import paho.mqtt.client
import paho.mqtt.publish
import numpy as np
import random
import datetime
import json
import ssl
import sys
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

# Conexión al broker

In [2]:
host = "broker.hivemq.com"

client = paho.mqtt.client.Client("Publicador de estantes", False)

client.qos = 2

client.connect(host=host)

0

# Conexión a la bd

In [4]:
myConnection = psycopg2.connect(
    host='lallah.db.elephantsql.com',
    user ='okgedrkq',
    password='OWJG1VssXGsfuUWTguGQoCmtZ5c5JZIp',
    dbname='okgedrkq'
)

In [5]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

# Usaremos POO para manejar los estantes

In [6]:
class Estante:
    producto = 0
    def __init__(self, id, id_sucursal, capacidad, disponibilidad):
        self.id = id
        self.id_sucursal = id_sucursal
        self.capacidad = capacidad
        self.disponibilidad = disponibilidad

# Traer información de la  BD

In [7]:
# Sucursal 1
query = """ select * from Sucursal
            Where id = 1
"""
sucursal_1 = select(query)
sucursal_1 = sucursal_1[0]

# Sucursal 2
query = """ select * from Sucursal
            Where id = 2
"""
sucursal_2 = select(query)
sucursal_2 = sucursal_2[0]

# Estantes de la sucursal 1
query = """ 
    select * from estantes
    where id_sucursal = 1
"""
estantes_1_DB = select(query)

# Estantes de la sucursal 2
query = """ 
    select * from estantes
    where id_sucursal = 2
"""
estantes_2_DB = select(query)

# Stock de la sucursal 1
query = """ 
    select * from stock
    where id_sucursal = 1
"""
stock_1 = select(query)

# Stock de la sucursal 2
query = """ 
    select * from stock
    where id_sucursal = 2
"""
stock_2 = select(query)

#El id de todos nuestros clientes registrados
query = """ select id from cliente
"""
clients = select(query)
clients = [row[0] for row in clients]
print(clients)

#El id de cliente de los clientes en el club
query = """ select id_cliente from club_clientes
"""
club_clients = select(query)
club_clients = [row[0] for row in club_clients]
print(club_clients)

#Traemos los preSio actualizados
query = """ 
with orden as (select * from precio order by fecha desc)
select distinct on (idproducto) idproducto as Producto,
monto
from orden
order by idproducto asc
"""
precios = select(query)
#precios = [row[0] for row in precios,row[1] for row in precios]
print(precios)
print(precios[0][1])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[1, 2]
[(1, 500000.0), (2, 9.0), (3, 30.0)]
500000.0


# Funciones auxliares

In [8]:
def insertVisit(date, sucursal, id):
    theDate = date.timestamp()
    query = f""" INSERT INTO visitas(fecha, id_sucursal, idcliente)
    VALUES(to_timestamp ( {theDate} ),{sucursal}, {id});
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    print("nueva visita")
    
def insertUnknownVisit(date, sucursal):
    theDate = date.timestamp()
    query = f""" INSERT INTO visitas(fecha, id_sucursal)
    VALUES(to_timestamp ( {theDate} ),{sucursal});
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    print("nueva visita")
    
def incrementVisit(id):
    query = f""" 
        UPDATE club_clientes 
           SET visitas = visitas + 1
        WHERE id_cliente = {id};
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    print("Una más al club");
    
def actualizarEstante(estante, producto, cantidad, date):
    theDate = date.timestamp()
    query = f""" INSERT INTO unidades(id_producto, id_estante, cantidad, fecha)
    VALUES({producto}, {estante}, {cantidad} ,to_timestamp ( {theDate} ));
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    
def llenarEstante(estante, producto, date):
    estante.producto = producto
    estante.disponibilidad = 0
    actualizarEstante(estante.id, producto, estante.capacidad, date)

def restockEstante(estante, producto, cantidad, date):
    theDate = date.timestamp()
    query = f""" INSERT INTO restock_estantes(id_producto, id_estante, cantidad, fecha)
    VALUES({producto}, {estante}, {cantidad} ,to_timestamp ( {theDate} ));
    """
    cur = myConnection.cursor()
    cur.execute(query)
    myConnection.commit()
    
def enviarAlerta(estante, sucursal):
    payload={
            "mensaje": f"el estante {estante} en la sucursal {sucursal} necesita refill",
            }        
    print (payload)
    client.publish('alertas/estantes',json.dumps(payload),qos=2)
    
def insertClient(cedula, nombre, apellido):
    query = f""" INSERT INTO cliente(cedula, nombre, apellido)
    VALUES({cedula}, '{nombre}', '{apellido}')
    RETURNING id;
    """
    cur = myConnection.cursor()
    cur.execute(query)
    id_client = cur.fetchone()[0]
    myConnection.commit()
    print("nuevo cliente registrado")
    return id_client

# Llenar estantes

In [9]:
# LLenar estantes de la sucursal 1
estantes_1 = []

#Dejamos la disponibilidad en cero para que estén llenos
for index, tuple in enumerate(estantes_1_DB):
    estante = Estante(tuple[0], tuple[1], tuple[2], tuple[3])
    estantes_1.append(estante)

for i in range(len(estantes_1)):
    random_product = random.randint(0, len(stock_1)-1)
    llenarEstante(estantes_1[i], stock_1[random_product][1], datetime.datetime.now())

    
# LLenar estantes de la sucursal 2
estantes_2 = []
for index, tuple in enumerate(estantes_2_DB):
    estante = Estante(tuple[0], tuple[1], tuple[2], tuple[3])
    estantes_2.append(estante)
    
for i in range(len(estantes_2)):
    random_product = random.randint(0, len(stock_2)-1)
    llenarEstante(estantes_2[i], stock_2[random_product][1], datetime.datetime.now())

print(estantes_1[4].disponibilidad)
print(estantes_2[2].disponibilidad)


0
0


# Definir variables principales

In [10]:
#Máximo de personas en las sucursales
max_personas_1 = sucursal_1[2]
max_personas_2 = sucursal_2[2]

#Contador de las personas en las sucursales
cont_personas_1 = 0
cont_personas_2 = 0

#Compradores en los pasillos
compradores_sucursal_1 = []
compradores_sucursal_2 = []

#Compradores en las colas
cola_sucursal_1 = []
cola_sucursal_2 = []

#Nombres random para los clientes nuevos
nombres_random = ["George", "John", "Timmy", "Grisselba", "Gabriella", "Constantinopla", "Jeferson", "Alvin", "Arthur", "Ocus"]
apellidos_random = ["Constanza", "Wick", "Testubinochi", "Larez", "Herrera", "Jackson", "Gutierritos", "Yakitori", "Fleck", "Pocus" ]

# Ciclo de la simulación

In [12]:
#Fecha de hoy
actualDate = datetime.datetime.now()
while(True):
    
    #Empieza un nuevo día a las 7 am
    print("--------------------- Nuevo Día ---------------------")
    actualDate = actualDate.replace(hour=7,minute=0,second=0)
    print(actualDate)
    
    #Sucursales vacías
    cont_personas_1 = 0
    cont_personas_2 = 0
    
    compradores_sucursal_1 = []
    compradores_sucursal_2 = []
    
    cola_sucursal_1 = []
    cola_sucursal_2 = []
    
    
    #cerramos a las 6 de la tarde
    while(actualDate.hour < 18):
        print(actualDate)
        #----Empezamos con la sucursal 1 
        print("--------------------- Sucursal 1 ---------------------")
        #------ Generamos una nueva entrada de un cliente si hay espacio
        if(cont_personas_1 <= max_personas_1):
            cont_personas_1 +=1
            #generamos la identidad del cliente con un número random
            random_id = random.randint(1, 10)
            
            persona = {
                "id":random_id,
                "productos":[],
                "carrito":[]
            }
            compradores_sucursal_1.append(persona)

            #Revisamos si el cliente es conocido
            if(random_id not in clients):
                #Añadimos la visita sin id
                insertUnknownVisit(actualDate, 1);
            else:  
                #añadimos la visita del cliente, con su id
                insertVisit(actualDate, 1, random_id);
            
        # Se decide si los clientes pasan a pagar o a buscar productos
        for i in range(len(compradores_sucursal_1)):
                #El tamaño de la lista cambia, hay que asegurarse de que no nos salgamos del rango
                if(i < len(compradores_sucursal_1)):
                    #Si el numero random es 1, enviamos el cliente a pagar, sino, a busacar un producto
                    decision = random.randint(1, 3)
                    
                    if(decision == 1 and len(compradores_sucursal_1[i]["productos"]) > 0):
                        print("---- a pagar ----")
                        cliente = compradores_sucursal_1.pop(i)
                        cola_sucursal_1.append(cliente)
                    else:
                        #estante random
                        random_estante = random.randint(0, len(estantes_1)-1)
                        #random_cantidad = random.randint(1, (estantes_1[random_estante].capacidad - estantes_1[random_estante].disponibilidad))
                        random_cantidad = random.randint(1, 15)
                        #cantidad random de productos
                        #capacidad - disponibilidad = cantidad de productos actuales
        
                        #Sólo metemos el producto al carrito si el cliente no lo ha metido aún, sino, no hacemos nada
                        if(estantes_1[random_estante].producto not in compradores_sucursal_1[i]["productos"]):
                        
                            #Metemos al carrito los productos
                            compradores_sucursal_1[i]["productos"].append(estantes_1[random_estante].producto)
                            conjunto = {
                                "producto":estantes_1[random_estante].producto,
                                "cantidad":random_cantidad
                            }
                            compradores_sucursal_1[i]["carrito"].append(conjunto)
                    
                            print("---- llevando productos ----")
        
                            #retiramos del estante la cantidad de productos que se llevaron
                            estantes_1[random_estante].disponibilidad += random_cantidad
                            cantidad_actual = estantes_1[random_estante].capacidad - estantes_1[random_estante].disponibilidad
                            actualizarEstante(estantes_1[random_estante].id, estantes_1[random_estante].producto, cantidad_actual, actualDate)
                            
                            #calculamos el límite para hacer refill
                            limite = (estantes_1[random_estante].capacidad*0.2)
        
                            if((estantes_1[random_estante].capacidad - estantes_1[random_estante].disponibilidad) <= limite):
                                print("ALERTA REFILL")
            
                                #Publicamos con el protocolo mqtt
                                enviarAlerta(estantes_1[random_estante].id, 1)
                                llenarEstante(estantes_1[random_estante], estantes_1[random_estante].producto, actualDate)
                                restockEstante(estantes_1[random_estante].id, estantes_1[random_estante].producto, estantes_1[random_estante].capacidad ,actualDate)
        
        #Los clientes en cola pagan
        while(len(cola_sucursal_1)>0):
            comprador = cola_sucursal_1.pop(0)
            #Si no es un comprador registrado lo añadimos a la BD
            print("id del comprador",comprador["id"])
            if(comprador["id"] not in clients):
                #Asignamos un nombre random al comprador
                nombre = nombres_random[random.randint(0, len(nombres_random)-1)]
                apellido = apellidos_random[random.randint(0, len(apellidos_random)-1)]
                cedula = random.randint(6000000, 30000000)
                
                id_client = insertClient(cedula, nombre, apellido)
                
                print(f"id:{id_client} {nombre} {apellido} ")
                
                comprador["id"] = id_client
                
                #actualizamos nuestra lista de clientes
                query = """ select id from cliente
                """
                clients = select(query)
                clients = [row[0] for row in clients]
                
            cont_personas_1 -=1
            
            #invocamos la función compra de al base de datos
            timeStamp = actualDate.timestamp()
            
            #reiniciamos el monto
            monto = 0
            #llenamos un arreglo a parte para el carrito del cliente
            car = []
            for i in comprador['carrito']:
                car.append((i['producto'], i['cantidad']))
                
                #Calculamos el monto total
                indiceProducto = [y[0] for y in precios].index(i['producto'])
                monto += (precios[indiceProducto][1]*i['cantidad'])
            print(car);
            print(monto)
            
            #Generar un banco random
            bancoRandom = random.randint(1, 3)
            if(bancoRandom == 1):
                bancoRandom = 'Mercantil'
            elif (bancoRandom == 2):
                bancoRandom = 'Provincial'
            else:
                bancoRandom = 'Banesco'
            
            query = f""" SELECT compraGustavo(array{car}::carrito[], {comprador["id"]}, '{bancoRandom}'::text, {timeStamp}, 1, {monto})"""
            
            cur = myConnection.cursor()
            cur.execute(query)
            cur.close()
            myConnection.commit()
            
            #Cada cliente tarda en promedio 6  minutos en comprar
            actualDate += datetime.timedelta(minutes=6)
        
        #----Seguimos con la sucursal 2
        print("--------------------- Sucursal 2 ---------------------")
        #------ Generamos una nueva entrada de un cliente si hay espacio
        if(cont_personas_2 <= max_personas_2):
            cont_personas_2 +=1
            #generamos la identidad del cliente con un número random
            random_id = random.randint(1, 10)
            
            persona = {
                "id":random_id,
                "productos":[],
                "carrito":[]
            }
            compradores_sucursal_2.append(persona)

            #Revisamos si el cliente es conocido
            if(random_id not in clients):
                #Añadimos la visita sin id
                insertUnknownVisit(actualDate, 2);
            else:  
                #añadimos la visita del cliente, con su id
                insertVisit(actualDate, 2, random_id);
            
        # Se decide si los clientes pasan a pagar o a buscar productos
        for i in range(len(compradores_sucursal_2)):
                #El tamaño de la lista cambia, hay que asegurarse de que no nos salgamos del rango
                if(i < len(compradores_sucursal_2)):
                    #Si el numero random es 1, enviamos el cliente a pagar, sino, a busacar un producto
                    decision = random.randint(1, 3)
                    
                    if(decision == 1 and len(compradores_sucursal_2[i]["productos"]) > 0):
                        print("---- a pagar ----")
                        cliente = compradores_sucursal_2.pop(i)
                        cola_sucursal_2.append(cliente)
                    else:
                        #estante random
                        random_estante = random.randint(0, len(estantes_2)-1)
                        #random_cantidad = random.randint(1, (estantes_2[random_estante].capacidad - estantes_2[random_estante].disponibilidad))
                        random_cantidad = random.randint(1, 15)
                        #cantidad random de productos
                        #capacidad - disponibilidad = cantidad de productos actuales
        
                        #Sólo metemos el producto al carrito si el cliente no lo ha metido aún, sino, no hacemos nada
                        if(estantes_2[random_estante].producto not in compradores_sucursal_2[i]["productos"]):
                        
                            #Metemos al carrito los productos
                            compradores_sucursal_2[i]["productos"].append(estantes_2[random_estante].producto)
                            conjunto = {
                                "producto":estantes_2[random_estante].producto,
                                "cantidad":random_cantidad
                            }
                            compradores_sucursal_2[i]["carrito"].append(conjunto)
                    
                            print("---- llevando productos ----")
        
                            #retiramos del estante la cantidad de productos que se llevaron
                            estantes_2[random_estante].disponibilidad += random_cantidad
                            cantidad_actual = estantes_2[random_estante].capacidad - estantes_2[random_estante].disponibilidad
                            actualizarEstante(estantes_2[random_estante].id, estantes_2[random_estante].producto, cantidad_actual, actualDate)
                            
                            #calculamos el límite para hacer refill
                            limite = (estantes_2[random_estante].capacidad*0.2)
        
                            if((estantes_2[random_estante].capacidad - estantes_2[random_estante].disponibilidad) <= limite):
                                print("ALERTA REFILL")
            
                                #Publicamos con el protocolo mqtt
                                enviarAlerta(estantes_2[random_estante].id, 2)
                                llenarEstante(estantes_2[random_estante], estantes_2[random_estante].producto, actualDate)
                                restockEstante(estantes_2[random_estante].id, estantes_2[random_estante].producto, estantes_2[random_estante].capacidad ,actualDate)
        
        #Los clientes en cola pagan
        while(len(cola_sucursal_2)>0):
            comprador = cola_sucursal_2.pop(0)
            #Si no es un comprador registrado lo añadimos a la BD
            print("id del comprador",comprador["id"])
            if(comprador["id"] not in clients):
                #Asignamos un nombre random al comprador
                nombre = nombres_random[random.randint(0, len(nombres_random)-1)]
                apellido = apellidos_random[random.randint(0, len(apellidos_random)-1)]
                cedula = random.randint(6000000, 30000000)
                
                id_client = insertClient(cedula, nombre, apellido)
                
                print(f"id:{id_client} {nombre} {apellido} ")
                
                comprador["id"] = id_client
                
                #actualizamos nuestra lista de clientes
                query = """ select id from cliente
                """
                clients = select(query)
                clients = [row[0] for row in clients]
                
            cont_personas_2 -=1
            
            #invocamos la función compra de al base de datos
            timeStamp = actualDate.timestamp()
            
            #reiniciamos el monto
            monto = 0
            #llenamos un arreglo a parte para el carrito del cliente
            car = []
            for i in comprador['carrito']:
                car.append((i['producto'], i['cantidad']))
                
                #Calculamos el monto total
                indiceProducto = [y[0] for y in precios].index(i['producto'])
                monto += (precios[indiceProducto][1]*i['cantidad'])
            print(car);
            print(monto)
            
            #Generar un banco random
            bancoRandom = random.randint(1, 3)
            if(bancoRandom == 1):
                bancoRandom = 'Mercantil'
            elif (bancoRandom == 2):
                bancoRandom = 'Provincial'
            else:
                bancoRandom = 'Banesco'
            
            query = f""" SELECT compraGustavo(array{car}::carrito[], {comprador["id"]}, '{bancoRandom}'::text, {timeStamp}, 2, {monto})"""
            
            cur = myConnection.cursor()
            cur.execute(query)
            cur.close()
            myConnection.commit()
            
            #Cada cliente tarda en promedio 6  minutos en comprar
            actualDate += datetime.timedelta(minutes=6)
        
        actualDate += datetime.timedelta(minutes=30)
        time.sleep(0.5)
   
    #Aumentamos la fecha un día
    actualDate += datetime.timedelta(days=1) 

--------------------- Nuevo Día ---------------------
2020-11-15 07:00:00.448516
2020-11-15 07:00:00.448516
--------------------- Sucursal 1 ---------------------
nueva visita
---- llevando productos ----
--------------------- Sucursal 2 ---------------------
nueva visita
---- llevando productos ----
2020-11-15 07:30:00.448516
--------------------- Sucursal 1 ---------------------
nueva visita
---- llevando productos ----
---- llevando productos ----
--------------------- Sucursal 2 ---------------------
nueva visita
---- llevando productos ----
2020-11-15 08:00:00.448516
--------------------- Sucursal 1 ---------------------
nueva visita
---- llevando productos ----
---- llevando productos ----
ALERTA REFILL
{'mensaje': 'el estante 3 en la sucursal 1 necesita refill'}
--------------------- Sucursal 2 ---------------------
nueva visita
---- a pagar ----
id del comprador 2
[(2, 3)]
27.0
2020-11-15 08:36:00.448516
--------------------- Sucursal 1 ---------------------
nueva visita
---- a

KeyboardInterrupt: 